## LAB SHEET 8

Neural networks are trained and tested with different datasets to make sure that the networks can actually
deal with new data and not just data it has already seen. If the label of a tested image corresponds to the
largest entry of the output vector, the image has been correctly classified by the network. The fraction of
correctly classified test images is a measure for the network’s performance.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import os
from PIL import Image

def sigmoid(x):  # Sigmoid Function
    return 1 / (1 + np.exp(-x))

class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.inodes = input_nodes
        self.hnodes = hidden_nodes
        self.onodes = output_nodes
        self.lr = learning_rate

        # Initialize weights
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))

    def train(self, inputs_list, targets_list):
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T

        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = sigmoid(hidden_inputs)

        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = sigmoid(final_inputs)

        output_errors = targets - final_outputs
        hidden_errors = np.dot(self.who.T, output_errors)

        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))

    def query(self, inputs_list):
        inputs = np.array(inputs_list, ndmin=2).T

        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = sigmoid(hidden_inputs)

        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = sigmoid(final_inputs)

        return final_outputs
    
    def saveWeights(self):
        np.save('wih.npy', self.wih)
        np.save('who.npy', self.who)

    def restoreWeights(self):
        self.wih = np.load('wih.npy')
        self.who = np.load('who.npy')


    def evaluate(self, data_test, labels_test):
        scorecard = []
        for i in range(len(data_test)):
            correct_label = np.argmax(labels_test[i])
            input_data = data_test[i].reshape(784)  # Flatten the image data
            outputs = self.query(input_data)
            predicted_label = np.argmax(outputs)
            scorecard.append(1 if predicted_label == correct_label else 0)
        return sum(scorecard) / len(scorecard)


def load_mnist_images_labels(images_folder, labels_file):
    image_files = sorted([f for f in os.listdir(images_folder) if f.endswith('.gif')])
    image_data = []
    for image_file in image_files:
        image_path = os.path.join(images_folder, image_file)
        with Image.open(image_path) as img:
            img_array = np.asarray(img, dtype=np.float32)
            image_data.append(img_array.flatten())
    image_data = np.array(image_data) / 255.0  # Normalize pixel values

    with open(labels_file, 'r') as f:
        labels = np.array([int(label.strip()) for label in f.readlines()])
    labels_one_hot = np.eye(10)[labels]

    return image_data, labels_one_hot

# Define paths
base_folder = 'lab_7/Resources/Extracted_Dataset/'
mnist_test_folder = os.path.join(base_folder, 'mnist_test')
labels_file = os.path.join(mnist_test_folder, 'labels.txt')

# Load the dataset from .gif files and labels.txt
data_images, labels = load_mnist_images_labels(mnist_test_folder, labels_file)

# Initialize the Neural Network
input_nodes = 784
hidden_nodes = 100
output_nodes = 10
learning_rate = 0.3

n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

# Train the Neural Network with the dataset
for i in range(len(data_images)):
    inputs = data_images[i]  # Assuming the data is already normalized
    targets = labels[i]
    n.train(inputs, targets)

# Evaluate the Neural Network with the same dataset (for demonstration purposes, normally you'd use a separate test set)
performance = n.evaluate(data_images, labels)
print("Performance:", performance)

# Visualize the Weight Matrices of the trained network
plt.imshow(n.wih, cmap='Greys', interpolation='None')
plt.title('Input-Hidden Weights')
plt.show()

plt.imshow(n.who, cmap='Greys', interpolation='None')
plt.title('Hidden-Output Weights')
plt.show()